# exp002  
データセットの作成（プロト）  
コードはスクリプト化する。  
データセット名は`dataset001`とし、本リポジトリ`data`直下に配置する。  
[Notion](https://www.notion.so/exp002-bb2a83196eba4feea0ae273917540f54?pvs=4)

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
import warnings
from tqdm import tqdm

%matplotlib inline
warnings.filterwarnings("ignore")

# リポジトリtopに移動
while os.path.basename(os.getcwd()) != 'rsna-2023':
    os.chdir('../')
    if os.getcwd() == '/':
        raise Exception('Could not find project root directory.')
    
from src.data_io import load_dicom_series

In [2]:
df_train = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train.csv')
df_train_image_level = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/image_level_labels.csv')
df_train_serirs_meta = pd.read_csv('data/rsna-2023-abdominal-trauma-detection/train_series_meta.csv')

base_dir = "data/rsna-2023-abdominal-trauma-detection"
dataset_dir = "data/dataset001"

In [3]:
len(df_train_serirs_meta)

4711

# 全dicom画像をHU値で.npy形式(int16)に変換・保存  

In [4]:
for idx, (pid, sid) in tqdm(enumerate(zip(df_train_serirs_meta["patient_id"], df_train_serirs_meta["series_id"]))):
    series_path = os.path.join(base_dir, "train_images", str(pid), str(sid))
    image_arr, path_list, meta_list = load_dicom_series(series_path)
    output_dir = os.path.join(dataset_dir, "train_images", str(pid), str(sid))
    os.makedirs(output_dir, exist_ok=True)
    for idx, path in enumerate(path_list):
        path = path.replace(".dcm", ".npy")
        np.save(os.path.join(output_dir, path), image_arr[idx])

745it [2:18:58,  6.12s/it]